In [ ]:
import numpy as np
from datetime import datetime, timedelta
from mapper_functions import plot_global_tight
from mapper_functions import plot_global

from my_functions import read_ascat_bufr_v2



In [ ]:
sat_name = 'M01-ASCA-ASCSMO02-NA-5.0-'
start_date = datetime(2015, 8, 1)
end_date = datetime(2015, 9, 1)
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

printflag = False

lat_list = []
lon_list = []
ssom_list = []
tpcx_list = []
domo_list = []
smpf_list = []
smcf_list = []
alfr_list = []
iwfr_list = []
snoc_list = []
flsf_list = []

# Loop over the dates
current_date = start_date
while current_date <= end_date:   
    # Format the date as a string
    year_month = current_date.strftime('%Y/M%m')
    
    # Use the formatted date in the path
    path = f'../test_data/Metop_B/Y{year_month}/'

    # Define the file name for the current date
    file_name_start = sat_name + current_date.strftime('%Y%m%d')

    # print('path = ', path)
    print('file_name_start = ', file_name_start)

    lat, lon, ssom, tpcx, domo, smpf, smcf, alfr, iwfr, snoc, flsf = read_ascat_bufr_v2(path, file_name_start, printflag)

    lat_list.append(lat)
    lon_list.append(lon)
    ssom_list.append(ssom)
    tpcx_list.append(tpcx)
    domo_list.append(domo)
    smpf_list.append(smpf)
    smcf_list.append(smcf)
    alfr_list.append(alfr)
    iwfr_list.append(iwfr)
    snoc_list.append(snoc)
    flsf_list.append(flsf)

    current_date += timedelta(days=1)




In [ ]:
# Concatenate the lists
lat = np.concatenate(lat_list)
lon = np.concatenate(lon_list)
ssom = np.concatenate(ssom_list)
tpcx = np.concatenate(tpcx_list)
domo = np.concatenate(domo_list)
smpf = np.concatenate(smpf_list)
smcf = np.concatenate(smcf_list)
alfr = np.concatenate(alfr_list)
iwfr = np.concatenate(iwfr_list)
snoc = np.concatenate(snoc_list)
flsf = np.concatenate(flsf_list)

In [ ]:
# Save the data
np.savez('ascat_data_201508.npz', lat=lat, lon=lon, ssom=ssom, tpcx=tpcx, domo=domo, smpf=smpf, smcf=smcf, alfr=alfr, iwfr=iwfr, snoc=snoc, flsf=flsf)

In [ ]:
path = '../test_data/Metop_B/Y2015/M08/'  # '/discover/nobackup/amfox/ASCAT_EUMETSAT/Metop-B/Y2019/M06' 
file_name_start = 'M01-ASCA-ASCSMO02-NA-5.0-20150801' # First ten days
printflag = True

lat, lon, ssom, tpcx, domo, smpf, smcf, alfr, iwfr, snoc, flsf = read_ascat_bufr_v2(path, file_name_start, printflag)

In [ ]:
# Save the variables in .npz file
np.savez(file_name_start, lat=lat, lon=lon, ssom=ssom, tpcx=tpcx, domo=domo, smpf=smpf, smcf=smcf, alfr=alfr, iwfr=iwfr, snoc=snoc, flsf=flsf)

In [ ]:
data = np.load('M01-ASCA-ASCSMO02-NA-5.0-2015080.npz' , allow_pickle=True)

# List the variables in the file
print(data.files)

In [ ]:

qc_lats = np.squeeze(data['lat'])
qc_lons = np.squeeze(data['lon'])
ssom = np.squeeze(data['ssom'])
smpf = np.squeeze(data['smpf'])
smcf = np.squeeze(data['smcf'])

mask = (smpf == 0)
land_lats = qc_lats[mask]
land_lons = qc_lons[mask]
land_ssom = ssom[mask]
land_smpf = smpf[mask]
land_smcf = smcf[mask]

mask = (smpf == 0) # & (smcf == 0)
ob_array = np.empty([np.sum(mask), 3])
ob_array.fill(np.nan)
ob_array[:, 1] = qc_lons[mask]
ob_array[:, 2] = qc_lats[mask]
ob_array[:, 0] = smcf[mask]

plot_global_tight(ob_array,False,'smcf','QC', 0, 8, cmap='viridis')

ob_array = np.empty([len(qc_lats), 3])
ob_array.fill(np.nan)
ob_array[:, 1] = qc_lons
ob_array[:, 2] = qc_lats
ob_array[:, 0] = smcf

plot_global_tight(ob_array,False,'smcf','QC', 0, 8, cmap='viridis')

In [ ]:
dlon = 0.25  # Longitude grid spacing
dlat = 0.25  # Latitude grid spacing

ll_lon = -180  # Lower left corner longitude
ll_lat = -90  # Lower left corner latitude

# Create a regular grid
grid_lon = np.arange(ll_lon, ll_lon + 360, dlon)
grid_lat = np.arange(ll_lat, ll_lat + 180, dlat)

# Calculate i,j values for each observation
i_cell = np.floor((qc_lons - ll_lon) / dlon).astype(int)
j_cell = np.floor((qc_lats - ll_lat) / dlat).astype(int)

# Combine i_cell and j_cell into a 2D array
combined = np.vstack((i_cell, j_cell)).T

# Find the unique pairs and their counts
unique_pairs1, counts = np.unique(combined, axis=0, return_counts=True)

# The counts array now contains the number of occurrences of each unique pair
obs_counts = counts

ob_array = np.empty([len(unique_pairs1), 3])
ob_array.fill(np.nan)
ob_array[:, 1] = grid_lon[unique_pairs1[:, 0]]
ob_array[:, 2] = grid_lat[unique_pairs1[:, 1]]
ob_array[:, 0] = obs_counts

plot_global_tight(ob_array,True,'Obs counts before QC (2015080*)','Count')



In [ ]:
# Calculate i,j values for each observation
i_cell = np.floor((land_lons - ll_lon) / dlon).astype(int)
j_cell = np.floor((land_lats - ll_lat) / dlat).astype(int)

# Combine i_cell and j_cell into a 2D array
combined = np.vstack((i_cell, j_cell)).T

# Find the unique pairs and their counts
unique_pairs1, counts = np.unique(combined, axis=0, return_counts=True)

# The counts array now contains the number of occurrences of each unique pair
obs_counts = counts

ob_array = np.empty([len(unique_pairs1), 3])
ob_array.fill(np.nan)
ob_array[:, 1] = grid_lon[unique_pairs1[:, 0]]
ob_array[:, 2] = grid_lat[unique_pairs1[:, 1]]
ob_array[:, 0] = obs_counts

plot_global_tight(ob_array,True,'Obs counts after QC (2015080*)','Count')

In [ ]:
print('len(unique_pairs1) = ', len(unique_pairs1), len(land_lats), len(obs_counts))

In [ ]:
sat_name = 'M01-ASCA-ASCSMO02-NA-5.0-'
start_date = datetime(2015, 8, 1)
end_date = datetime(2015, 8, 31)
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

# Calculate number of days between start and end date
num_days = (end_date - start_date).days + 1

printflag = False

dlon = 0.25  # Longitude grid spacing
dlat = 0.25  # Latitude grid spacing

ll_lon = -180  # Lower left corner longitude
ll_lat = -90  # Lower left corner latitude

# Create a regular grid
grid_lon = np.arange(ll_lon, ll_lon + 360, dlon)
grid_lat = np.arange(ll_lat, ll_lat + 180, dlat)

ob_array = np.empty([num_days, len(grid_lat), len(grid_lon)])
ob_array.fill(0)
land_ob_array = np.empty([num_days, len(grid_lat), len(grid_lon)])
land_ob_array.fill(0)

# Loop over the dates
current_date = start_date
day_of_run = 0

while current_date <= end_date:   
    # Format the date as a string
    year_month = current_date.strftime('%Y/M%m')
    
    # Use the formatted date in the path
    path = f'../test_data/Metop_B/Y{year_month}/'

    # Define the file name for the current date
    file_name_start = sat_name + current_date.strftime('%Y%m%d')

    # print('path = ', path)
    print('file_name_start = ', file_name_start)

    lat, lon, ssom, tpcx, domo, smpf, smcf, alfr, iwfr, snoc, flsf = read_ascat_bufr_v2(path, file_name_start, printflag)

    lat_in = np.squeeze(lat)
    lon_in = np.squeeze(lon)
    smpf_in = np.squeeze(smpf)

    mask = (smpf_in == 0)
    land_lats = lat_in[mask]
    land_lons = lon_in[mask]

    # Calculate i,j values for each observation
    i_cell = np.floor((lon_in - ll_lon) / dlon).astype(int)
    j_cell = np.floor((lat_in - ll_lat) / dlat).astype(int)

    # Combine i_cell and j_cell into a 2D array
    combined = np.vstack((i_cell, j_cell)).T

    # Find the unique pairs and their counts
    unique_pairs1, counts = np.unique(combined, axis=0, return_counts=True)

    # The counts array now contains the number of occurrences of each unique pair
    obs_counts = counts

    for i in range(len(unique_pairs1)):
        # Add the observation count to the corresponding grid cell
        obs_count = obs_counts[i]
        i_c = unique_pairs1[i, 0]
        j_c = unique_pairs1[i, 1]
        ob_array[day_of_run, j_c, i_c] = obs_count

    # Repeat for the land only data
    i_cell = np.floor((land_lons - ll_lon) / dlon).astype(int)
    j_cell = np.floor((land_lats - ll_lat) / dlat).astype(int)
    combined = np.vstack((i_cell, j_cell)).T
    unique_pairs1, counts = np.unique(combined, axis=0, return_counts=True)
    obs_counts = counts

    for i in range(len(unique_pairs1)):
        obs_count = obs_counts[i]
        i_c = unique_pairs1[i, 0]
        j_c = unique_pairs1[i, 1]
        land_ob_array[day_of_run, j_c, i_c] = obs_count

    day_of_run += 1
    current_date += timedelta(days=1)

# Save the data
np.savez('ascat_data_201508.npz', ob_array=ob_array, land_ob_array=land_ob_array, grid_lon=grid_lon, grid_lat=grid_lat)


In [ ]:
# Load the data
data = np.load('ascat_data_201508.npz', allow_pickle=True)
ob_array = data['ob_array']
land_ob_array = data['land_ob_array']
grid_lon = data['grid_lon']
grid_lat = data['grid_lat']

In [ ]:
# Flatten ob_array and then plot with plot_global_tight
ob_array_flat = ob_array[0, :, :].flatten(order='F')
# Replace 0 with nan in ob_array_flat
ob_array_flat[ob_array_flat == 0] = np.nan
lats_flat = np.tile(grid_lat, len(grid_lon))
lons_flat = np.repeat(grid_lon, len(grid_lat))

obarray = np.empty([len(ob_array_flat), 3])
obarray.fill(np.nan)
obarray[:, 1] = lons_flat
obarray[:, 2] = lats_flat
obarray[:, 0] = ob_array_flat

plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs before QC (20150801)','Count', 1, 10)

# Flatten ob_array and then plot with plot_global_tight
ob_array_flat = ob_array[1, :, :].flatten(order='F')
ob_array_flat[ob_array_flat == 0] = np.nan
obarray[:, 0] = ob_array_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs before QC (20150802)','Count', 1, 10)

# Flatten land_ob_array and then plot with plot_global_tight
land_ob_array_flat = land_ob_array[0, :, :].flatten(order='F')
# Replace 0 with nan in land_ob_array_flat
land_ob_array_flat[land_ob_array_flat == 0] = np.nan
obarray[:, 0] = land_ob_array_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs after QC (20150801)','Count', 1, 6)

# Flatten land_ob_array and then plot with plot_global_tight
land_ob_array_flat = land_ob_array[1, :, :].flatten(order='F')
# Replace 0 with nan in land_ob_array_flat
land_ob_array_flat[land_ob_array_flat == 0] = np.nan
obarray[:, 0] = land_ob_array_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs after QC (20150802)','Count', 1, 6)



In [ ]:
# Sum ob_array along the time axis
total_obs = np.sum(ob_array, axis=0)
land_total_obs = np.sum(land_ob_array, axis=0)

# Flatten total_obs and then plot with plot_global_tight
total_obs_flat = total_obs.flatten(order='F')
total_obs_flat[total_obs_flat == 0] = np.nan

obarray[:, 0] = total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs before QC (201508)','Count')

# Flatten land_total_obs and then plot with plot_global_tight
land_total_obs_flat = land_total_obs.flatten(order='F')
land_total_obs_flat[land_total_obs_flat == 0] = np.nan
obarray[:, 0] = land_total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs after QC (201508)','Count')

# Sum ob_array along the first 5 arrays on the time axis
total_obs = np.sum(ob_array[:5, :, :], axis=0)
land_total_obs = np.sum(land_ob_array[:5, :, :], axis=0)

# Flatten total_obs and then plot with plot_global_tight
total_obs_flat = total_obs.flatten(order='F')
total_obs_flat[total_obs_flat == 0] = np.nan
obarray[:, 0] = total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs before QC (20150801 - 20150805)','Count')

# Flatten land_total_obs and then plot with plot_global_tight
land_total_obs_flat = land_total_obs.flatten(order='F')
land_total_obs_flat[land_total_obs_flat == 0] = np.nan
obarray[:, 0] = land_total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs after QC (20150801 - 20150805)','Count')

# Sum ob_array along the first 5 arrays on the time axis
total_obs = np.sum(ob_array[:10, :, :], axis=0)
land_total_obs = np.sum(land_ob_array[:10, :, :], axis=0)

# Flatten total_obs and then plot with plot_global_tight
total_obs_flat = total_obs.flatten(order='F')
total_obs_flat[total_obs_flat == 0] = np.nan
obarray[:, 0] = total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs before QC (20150801 - 20150810)','Count')

# Flatten land_total_obs and then plot with plot_global_tight
land_total_obs_flat = land_total_obs.flatten(order='F')
land_total_obs_flat[land_total_obs_flat == 0] = np.nan
obarray[:, 0] = land_total_obs_flat
plot_global(obarray,True,'Total M01-ASCA-ASCSMO02-NA-5.0 obs after QC (20150801 - 20150810)','Count')